In [11]:
# Import library-library
import os
import glob

# Data Preparation and Preprocessing
import pandas as pd
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
NLTK_StopWords = stopwords.words('indonesian')
NLTK_StopWords = set(NLTK_StopWords)

# Word Embedding
import joblib
from sklearn.feature_extraction.text import TfidfVectorizer
import scipy.sparse

from sklearn.metrics.pairwise import cosine_similarity

# Input and Expansion Query
from textblob import TextBlob
from nltk.tokenize import wordpunct_tokenize
#from googletrans import Translator

#Split to train and test
from sklearn.model_selection import train_test_split

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/syubbanfakhriya/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [39]:
def preprocessing(berita):
    s = berita.lower()
    s = s.replace('\n', ' ')
    s = s.replace('\r', ' ')
    s = re.sub(r'[^a-zA-Z0-9\s]', ' ', s)
    tokens = [token for token in s.split(" ") if token != ""]
    T = [t for t in tokens if t not in NLTK_StopWords]
    return T

In [40]:
df =pd.read_csv("news.csv")
df.head()

,title,date,description,source
0,\n 6 Kios Terminal Lama Bojonegoro lude...,"Jumat, 28 Jan 2022 09:24 WIB",Lokasi itu dikenal warga sebagai pasar ayam. D...,www.detik.com
1,"\n Gempa M 5,1 Guncang Enggano Bengkulu...","Sabtu, 26 Feb 2022 07:09 WIB","berkekuatan magnitudo (M) 5,1 terjadi di Engga...",www.detik.com
2,"\r\n Sumbar Dilanda Gempa, BUMN Ikut Tu...","Sabtu, 26 Feb 2022 18:58 WIB",Gempa mengguncang wilayah Sumatera Barat denga...,www.detik.com
3,\n Kandang Ayam di Blitar Ludes Terbaka...,"Jumat, 28 Jan 2022 12:26 WIB",Api terlihat mulai melahap bangunan dua lantai...,www.detik.com
4,"\n Tewaskan 3 Orang, Begini Penampakan ...","Sabtu, 29 Jan 2022 12:31 WIB","rumah di Kebon Baru, Tebet, Jakarta Selatan (J...",www.detik.com


In [41]:
column = ['Title', 'Date', 'Description', 'Source']
df_total = pd.DataFrame()

df =pd.read_csv("news.csv")
isi=df['description']
df = df.dropna(subset=['description'], axis=0)
for x in range(0, df.shape[0]):
    text = preprocessing (df.iloc[x, -2])
    df.iloc[x, -2] = (' '.join(text)) 

In [42]:
df.head()
df_train, df_test=train_test_split(df,test_size=0.1)
df_train.head()

,title,date,description,source
1650,\n Ketum PAN Zulhas Minta Kader PAN Ber...,"Senin, 18 Jan 2021 04:43 WIB",ketua menginstruksikan kader pan lokasi bencan...,www.detik.com
826,"\n Blitar Berpotensi Tsunami, BPBD Mint...","Jumat, 04 Jun 2021 08:09 WIB",deretan pantai pesisir blitar selatan wilayah ...,www.detik.com
884,\n Cerita Satu Dekade Kampung Beting Pa...,"Selasa, 18 Jun 2019 06:54 WIB",kampung beting desa pantai bahagia muara gembo...,www.detik.com
12,\n Hujan dan Angin Kencang Masih Mengan...,"Kamis, 13 Jan 2022 09:00 WIB",bmkg klas i juanda surabaya mengimbau masyarak...,www.detik.com
270,\n Atap Rumah Warga Trenggalek Runtuh D...,"Selasa, 18 Jan 2022 20:36 WIB",kapolsek panggul akp budi hartoyo bencana ruma...,www.detik.com


In [43]:
desc_text_train = []
desc_text_test = []

for x in range(0, df_train.shape[0]):
  desc_text_train.append(df_train.iloc[x, -2])

for x in range(0, df_test.shape[0]):
  desc_text_test.append(df_test.iloc[x, -2])

In [45]:
print(df_test.info())
print("Panjang Dokumen Test : ", len(desc_text_test))

print ('-'*90)

print(df_train.info())
print("Panjang Dokumen Train : ", len(desc_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 179 entries, 1415 to 769
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        179 non-null    object
 1   date         179 non-null    object
 2   description  179 non-null    object
 3   source       179 non-null    object
dtypes: object(4)
memory usage: 7.0+ KB
None
Panjang Dokumen Test :  179
------------------------------------------------------------------------------------------
<class 'pandas.core.frame.DataFrame'>
Int64Index: 1611 entries, 1650 to 1526
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1611 non-null   object
 1   date         1611 non-null   object
 2   description  1611 non-null   object
 3   source       1611 non-null   object
dtypes: object(4)
memory usage: 62.9+ KB
None
Panjang Dokumen Train :  1611


In [46]:
# Simpan teks asli dokumen
joblib.dump(desc_text_test, "corpus/model/desc_text_test.pkl")
joblib.dump(desc_text_train, "corpus/model/desc_text_train.pkl")

# Simpan dataframe hasil text processing sebelum TF-IDF Vectorizer
df.to_csv('corpus/dataset/df_total.csv',index=False )
df_test.to_csv('corpus/dataset/df_test.csv', index=False)
df_train.to_csv('corpus/dataset/df_train.csv', index=False)


In [47]:
# Fit transform menggunakan TF-IDF Vectorizer
tfidf_vectorizer = TfidfVectorizer()
tfidf_test_matrix = tfidf_vectorizer.fit_transform(desc_text_test)
tfidf_train_matrix = tfidf_vectorizer.fit_transform(desc_text_train)

print('tfidf_test_matrix')
print(tfidf_test_matrix.shape)
print('tfidf_train_matrix')
print(tfidf_train_matrix.shape)

tfidf_test_matrix
(179, 4843)
tfidf_train_matrix
(1611, 16508)


In [48]:
joblib.dump(tfidf_vectorizer, "corpus/vectorizer/vectorizer.pkl")
joblib.dump(tfidf_test_matrix, "corpus/matrix/tfidf_test.pkl")
joblib.dump(tfidf_train_matrix, "corpus/matrix/tfidf_train.pkl")

['corpus/matrix/tfidf_train.pkl']

### Pencarian 

In [50]:
df_total = pd.read_csv('corpus/dataset/df_total.csv')
df_total = df_total[pd.notnull(df_total['description'])]
print(df_total.info())
print ('-'*90)

document_text_train= joblib.load('corpus/model/desc_text_train.pkl')
document_text_test= joblib.load('corpus/model/desc_text_test.pkl')
print(len(document_text_test))
print(len(document_text_train))

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1790 entries, 0 to 1789
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   title        1790 non-null   object
 1   date         1790 non-null   object
 2   description  1790 non-null   object
 3   source       1790 non-null   object
dtypes: object(4)
memory usage: 69.9+ KB
None
------------------------------------------------------------------------------------------
179
1611


In [51]:
kueri='Banjir jakarta'
kueri=preprocessing(kueri)
kueri= [" ".join (kueri)]
print (kueri)

tfidf_matrix =joblib.load( "corpus/matrix/tfidf_train.pkl" )
tfidf_vectorizer = joblib.load( "corpus/vectorizer/vectorizer.pkl" ) 
query_vec= tfidf_vectorizer.transform(kueri)
results=cosine_similarity(tfidf_matrix, query_vec).reshape((-1))
#print (results)
print("\n======================\n")
print("Top 10 most similar documents in corpus:")

j = 1

for i in results.argsort()[-10:][::-1]:
    print(j)
    print("No ID Dokumen  : ", i)
    print("Tanggal        : ", df_total.iloc[i,1])
    print("Isi berita     : ", df_total.iloc[i,2])
    print("(Score: %.4f) " % results[i])
    j += 1

['banjir jakarta']


Top 10 most similar documents in corpus:
1
No ID Dokumen  :  972
Tanggal        :  Selasa, 16 Nov 2021 15:10 WIB
Isi berita     :  gubernur jawa barat ridwan kamil mengimbau masyarakat mewaspadai bencana banjir longsor maret 2022 mengarahkan bupati walikota siaga menghadapi potensi bencana siklus cuaca maret potensi kebencanaan banjir longsor arahkan walikota bupati posisinya siaga 1 maret la nina gelombang gelombang iklim memengaruhi tingginya potensi kebencanaan ridwan kamil hotel horison kota bandung selasa 16 11 2021 kang emil sapaan ridwan kamil mengarahkan bpbd provinsi jawa barat mengecek 40 titik rawan banjir daerah daerah daerah aliran sungai das menunggu bencana korban bereaksi masyarakat waspada bantaran sungai kang emil basarnas kansar bandung jabar quick response jqr melatih relawan kebencanaan 27 kabupaten kota jabar pelatihan menunjang kesiapsiagaan menghadapi potensi bencana alam jabar mengunjungi sman 1 lembang pagi kang emil mengeluarkan surat bup